In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy import wcs
from astroquery.svo_fps import SvoFps
import numpy as np
import reproject
from astropy.visualization import simple_norm
basepath = '/orange/adamginsburg/jwst/sgrb2/NB'
import os
import astropy.units as u

In [2]:
fh_405 = fits.open('/orange/adamginsburg/jwst/sgrb2/NB/F405N/pipeline/jw05365-o001_t001_nircam_clear-f405n-merged_i2d.fits')
fh_410 = fits.open('/orange/adamginsburg/jwst/sgrb2/NB/F410M/pipeline/jw05365-o001_t001_nircam_clear-f410m-merged_i2d.fits')
file = '/orange/adamginsburg/jwst/sgrb2/NB/F405_minus_F410cont_pipeline_v0.1.fits'
if os.path.exists(file):
    bra_minus_cont = fits.getdata(file)
    #header = fits.getheader(file)
    #ww = wcs.WCS(header)
else:

    ww405 = wcs.WCS(fh_405['SCI'].header)
    ww410 = wcs.WCS(fh_410['SCI'].header)
    instrument = fh_405[0].header['INSTRUME']
    telescope = fh_405[0].header['TELESCOP']
    filt405 = fh_405[0].header['PUPIL']
    wavelength_table_405 = SvoFps.get_transmission_data(f'{telescope}/{instrument}.{filt405}')
    filt410 = fh_410[0].header['FILTER']
    wavelength_table_410 = SvoFps.get_transmission_data(f'{telescope}/{instrument}.{filt410}')
    waves_410 = wavelength_table_410['Wavelength']
    trans_405 = np.interp(waves_410, wavelength_table_405['Wavelength'], wavelength_table_405['Transmission'])
    trans_410 = wavelength_table_410['Transmission']

    fractional_bandwidth_405 = ( (trans_410/trans_410.max()) * (trans_405/trans_405.max()) ).sum() / (trans_410/trans_410.max()).sum()

    data_405_proj_410 = reproject.reproject_exact(fh_405['SCI'], fh_410['SCI'].header)
    cont410_sub_bra = (fh_410['SCI'].data - data_405_proj_410[0]*fractional_bandwidth_405) / (1-fractional_bandwidth_405)
    #fits.PrimaryHDU(data=cont410_sub_bra, header=fh_410['SCI'].header).writeto(f'{basepath}/F410_minus_F405_fractional_bandwidth.fits', overwrite=True)
    bra_minus_cont = data_405_proj_410[0] - cont410_sub_bra #* fractional_bandwidth_405
    fits.PrimaryHDU(data=bra_minus_cont, header=fh_410['SCI'].header).writeto(f'{file}', overwrite=False)

In [3]:
fh_187 = fits.open('/orange/adamginsburg/jwst/sgrb2/NB/F187N/pipeline/jw05365-o001_t001_nircam_clear-f187n-merged_i2d.fits')
fh_182 = fits.open('/orange/adamginsburg/jwst/sgrb2/NB/F182M/pipeline/jw05365-o001_t001_nircam_clear-f182m-merged_i2d.fits')
file = '/orange/adamginsburg/jwst/sgrb2/NB/F187_minus_F182cont_pipeline_v0.1.fits'
if os.path.exists(file):
    paa_minus_cont = fits.getdata(file)

else:
    ww187 = wcs.WCS(fh_187['SCI'].header)
    ww182 = wcs.WCS(fh_182['SCI'].header)
    instrument = fh_187[0].header['INSTRUME']
    telescope = fh_187[0].header['TELESCOP']
    filt187 = fh_187[0].header['FILTER']
    wavelength_table_187 = SvoFps.get_transmission_data(f'{telescope}/{instrument}.{filt187}')
    filt182 = fh_182[0].header['FILTER']
    wavelength_table_182 = SvoFps.get_transmission_data(f'{telescope}/{instrument}.{filt182}')
    waves_182 = wavelength_table_182['Wavelength']
    trans_187 = np.interp(waves_182, wavelength_table_187['Wavelength'], wavelength_table_187['Transmission'])
    trans_182 = wavelength_table_182['Transmission']

    fractional_bandwidth_187 = ( (trans_182/trans_182.max()) * (trans_187/trans_187.max()) ).sum() / (trans_182/trans_182.max()).sum()

    data_187_proj_182 = reproject.reproject_exact(fh_187['SCI'], fh_182['SCI'].header)

    cont182_sub_187 = (fh_182['SCI'].data - data_187_proj_182[0]*fractional_bandwidth_187) / (1-fractional_bandwidth_187)
    #fits.PrimaryHDU(data=cont182_sub_187, header=fh_182['SCI'].header).writeto(f'{basepath}/F182_minus_F187_fractional_bandwidth_pipeline_v0.1.fits', overwrite=True)

    paa_minus_cont = data_187_proj_182[0] - cont182_sub_187 #* fractional_bandwidth_405
    fits.PrimaryHDU(data=paa_minus_cont, header=fh_182['SCI'].header).writeto(f'{basepath}/F187_minus_F182cont_pipeline_v0.1.fits', overwrite=Flase)

In [4]:
data_paa_proj_bra = reproject.reproject_exact((paa_minus_cont, fh_182['SCI'].header), fh_410['SCI'].header)

Set DATE-AVG to '2024-09-07T16:21:01.359' from MJD-AVG.
Set DATE-END to '2024-09-07T17:05:17.716' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to    -4.647443 from OBSGEO-[XYZ].
Set OBSGEO-H to 1298271102.952 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set DATE-AVG to '2024-09-07T16:21:01.379' from MJD-AVG.
Set DATE-END to '2024-09-07T17:05:17.716' from MJD-END'. [astropy.wcs.wcs]
/blue/adamginsburg/nbudaiev/python_env/JWST_SgrB2/lib/python3.12/site-packages/reproject/common.py:170: UserWarning: The reproject_exact function currently has precision issues with images that have resolutions below ~0.05 arcsec, so the results may not be accurate.
  return reproject_func(


In [5]:
paa_bra_ratio_BW_scaled = (data_paa_proj_bra[0] * 0.024 ) / (bra_minus_cont * 0.046) # values taken from https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-instrumentation/nircam-filters#NIRCamFilters-tablenotes

In [6]:
#From Draine 2011:
#At 5000K: 3.95
#at 10000K: 4.24
#at 20000K: 4.55
emissivity_ratio = 4.24

In [7]:
from dust_extinction.parameter_averages import CCM89, G23

In [8]:
ext = G23(Rv=3.1)
R_intrinsic = emissivity_ratio

In [9]:
wavelengths = np.array([1.87, 4.05])* u.micron
A_lambda_over_Av = ext(wavelengths)
k = A_lambda_over_Av[0] - A_lambda_over_Av[1]

In [10]:
A_V = 2.5 * np.log10(R_intrinsic / paa_bra_ratio_BW_scaled) / k

/scratch/local/6867921/ipykernel_3780274/1980414983.py:1: RuntimeWarning: invalid value encountered in log10
  A_V = 2.5 * np.log10(R_intrinsic / paa_bra_ratio_BW_scaled) / k


In [11]:
np.nanmax(A_V)  

177.14184008461908

In [12]:
A_K_over_AV = ext(2.2*u.micron) # using 2.2 microns to get the ratio of A_V and A_K
A_K = A_K_over_AV * A_V

In [13]:
np.nanmax(A_K)

17.998429842517094

In [14]:
def get_extinction(observed_ratio_BW_scaled):
    #ext = CCM89(Rv=3.1) 
    ext = G23(Rv=3.1) #CCM89 is not defined for wavelengths > 3.5 microns
    #From Draine 2011:
    #At 5000K: 3.95
    #at 10000K: 4.24
    #at 20000K: 4.55
    emissivity_ratio = 4.24
    R_intrinsic = emissivity_ratio
    
    wavelengths = np.array([1.87, 4.05]) * u.micron # important to specifcy units, otherwise "ext()" assumes inverse microns
    A_lambda_over_Av = ext(wavelengths)
    k = A_lambda_over_Av[0] - A_lambda_over_Av[1]
    A_V = 2.5 * np.log10(R_intrinsic / observed_ratio_BW_scaled) / k

    A_K_over_AV = ext(2.2*u.micron) # using 2.2 microns to get the ratio of A_V and A_K
    A_K = A_K_over_AV * A_V

    return A_V, A_K

In [15]:
A_V, A_K = get_extinction(paa_bra_ratio_BW_scaled)

/scratch/local/6867921/ipykernel_3780274/2958818747.py:14: RuntimeWarning: invalid value encountered in log10
  A_V = 2.5 * np.log10(R_intrinsic / observed_ratio_BW_scaled) / k


In [16]:
np.nanmax(A_V), np.nanmax(A_K)

(177.14184008461908, 17.998429842517094)

In [17]:
fits.PrimaryHDU(data=A_K, header=fh_410['SCI'].header).writeto(f'{basepath}/A_K_map_assuming_Cardelli_law.fits', overwrite=True)

In [ ]:
paa_emission_threhsold = 1.5 # MJy/sr -- determined by visual inspection of the Paa image
paa_minus_cont_masked = paa_minus_cont.copy()
paa_minus_cont_masked[paa_minus_cont_masked < paa_emission_threhsold]= np.nan
data_paa_proj_bra_masked = reproject.reproject_exact((paa_minus_cont_masked, fh_182['SCI'].header), fh_410['SCI'].header)

Set DATE-AVG to '2024-09-07T16:21:01.359' from MJD-AVG.
Set DATE-END to '2024-09-07T17:05:17.716' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to    -4.647443 from OBSGEO-[XYZ].
Set OBSGEO-H to 1298271102.952 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set DATE-AVG to '2024-09-07T16:21:01.379' from MJD-AVG.
Set DATE-END to '2024-09-07T17:05:17.716' from MJD-END'. [astropy.wcs.wcs]
/blue/adamginsburg/nbudaiev/python_env/JWST_SgrB2/lib/python3.12/site-packages/reproject/common.py:170: UserWarning: The reproject_exact function currently has precision issues with images that have resolutions below ~0.05 arcsec, so the results may not be accurate.
  return reproject_func(


In [19]:
paa_bra_ratio_BW_scaled_masked_paa = (data_paa_proj_bra_masked[0] * 0.024 ) / (bra_minus_cont * 0.046)

In [27]:
A_K_masked_paa = get_extinction(paa_bra_ratio_BW_scaled_masked_paa)[1]

/scratch/local/6867921/ipykernel_3780274/2958818747.py:14: RuntimeWarning: invalid value encountered in log10
  A_V = 2.5 * np.log10(R_intrinsic / observed_ratio_BW_scaled) / k


In [26]:
np.nanmax(A_K_masked_paa)

10.556998288704204

In [21]:
fits.PrimaryHDU(data=A_K_masked_paa, header=fh_410['SCI'].header).writeto(f'{basepath}/A_K_map_assuming_Cardelli_law_Paa_based_mask.fits', overwrite=True)

In [ ]:
bra_emission_threhsold = 3 # MJy/sr -- determined by visual inspection of the Bra image
bra_minus_cont_masked = bra_minus_cont.copy()
bra_minus_cont_masked[bra_minus_cont_masked < bra_emission_threhsold]= np.nan

In [23]:
paa_bra_ratio_BW_scaled_masked_bra = (data_paa_proj_bra[0] * 0.024 ) / (bra_minus_cont_masked * 0.046)
A_K_masked_bra = get_extinction(paa_bra_ratio_BW_scaled_masked_bra)[1]
fits.PrimaryHDU(data=A_K_masked_bra, header=fh_410['SCI'].header).writeto(f'{basepath}/A_K_map_assuming_Cardelli_law_Bra_based_mask.fits', overwrite=True)

/scratch/local/6867921/ipykernel_3780274/2958818747.py:14: RuntimeWarning: invalid value encountered in log10
  A_V = 2.5 * np.log10(R_intrinsic / observed_ratio_BW_scaled) / k


In [28]:
np.nanmax(A_K_masked_bra)

15.817447016839456